In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style= "darkgrid", color_codes = True)
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, Input
from keras.regularizers import l2
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy
from sklearn.metrics import roc_curve, auc
from tensorflow.keras.preprocessing.image import  load_img
import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import pickle

In [2]:
# Define image size and other parameters
img_width, img_height = 256, 256
batchsize = 32
epochs = 4
num_of_class = 2

validation = keras.utils.image_dataset_from_directory(
    directory='Resources/val',
    labels='inferred',
    label_mode='categorical',
    batch_size=batchsize,
    image_size=(img_width, img_height))

Found 16 files belonging to 2 classes.


In [3]:
test = keras. utils.image_dataset_from_directory(
    directory='Resources/test',
    labels='inferred',
    label_mode='categorical',
    batch_size=batchsize,
    image_size=(img_width, img_height))

Found 624 files belonging to 2 classes.


In [4]:
train = keras. utils.image_dataset_from_directory(
    directory='Resources/train',
    labels='inferred',
    label_mode='categorical',
    batch_size=batchsize,
    image_size=(img_width, img_height))

Found 5216 files belonging to 2 classes.


In [5]:
print(train.class_names)
print(validation.class_names)
print(test.class_names)

['NORMAL', 'PNEUMONIA']
['NORMAL', 'PNEUMONIA']
['NORMAL', 'PNEUMONIA']


In [6]:
x_train = []
y_train = []

for feature, label in train:
    x_train.append(feature.numpy())
    y_train.append(label.numpy())

x_train = np.concatenate(x_train, axis=0)
y_train = np.concatenate(y_train, axis=0)

In [7]:
x_val = []
y_val = []
    
for feature, label in validation:
    x_val.append(feature.numpy())
    y_val.append(label.numpy())

x_val = np.concatenate(x_val, axis=0)

y_val = np.concatenate(y_val, axis=0)

In [8]:
x_test = []
y_test = []

for feature, label in test:
    x_test.append(feature.numpy())
    y_test.append(label.numpy())

x_test = np.concatenate(x_test, axis=0)
y_test = np.concatenate(y_test, axis=0)

In [9]:
# check the shapes of 'x_train' and 'y_train':
print("Shape of 'x_train':", x_train.shape)
print("Shape of 'y_train':", y_train.shape)
print("Shape of 'x_val':", x_val.shape)
print("Shape of 'y_val':", y_val.shape)
print("Shape of 'x_test':", x_test.shape)
print("Shape of 'y_test':", y_test.shape)

Shape of 'x_train': (5216, 256, 256, 3)
Shape of 'y_train': (5216, 2)
Shape of 'x_val': (16, 256, 256, 3)
Shape of 'y_val': (16, 2)
Shape of 'x_test': (624, 256, 256, 3)
Shape of 'y_test': (624, 2)


In [10]:
x_train=x_train/255
x_val=x_val/255
x_test=x_test/255

In [11]:
y_train

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [12]:
x_train_pneumonia = []
x_train_normal = []

for i in range(len(x_train)):
    if y_train[i][0] == 0:
        x_train_pneumonia.append(list(x_train[i]))
    else:
        x_train_normal.append(list(x_train[i]))

x_train_pneumonia = np.array(x_train_pneumonia)
x_train_normal = np.array(x_train_normal)

In [13]:
print(np.shape(x_train_pneumonia))

(3875, 256, 256, 3)


In [14]:
x_train_pneumonia[0][0]

array([[0.1868681 , 0.1868681 , 0.1868681 ],
       [0.17781863, 0.17781863, 0.17781863],
       [0.17781863, 0.17781863, 0.17781863],
       [0.17144608, 0.17144608, 0.17144608],
       [0.17225413, 0.17225413, 0.17225413],
       [0.17316176, 0.17316176, 0.17316176],
       [0.16605392, 0.16605392, 0.16605392],
       [0.15833333, 0.15833333, 0.15833333],
       [0.15424709, 0.15424709, 0.15424709],
       [0.15294118, 0.15294118, 0.15294118],
       [0.14752987, 0.14752987, 0.14752987],
       [0.13833487, 0.13833487, 0.13833487],
       [0.14264706, 0.14264706, 0.14264706],
       [0.12860754, 0.12860754, 0.12860754],
       [0.11857767, 0.11857767, 0.11857767],
       [0.1254902 , 0.1254902 , 0.1254902 ],
       [0.12220052, 0.12220052, 0.12220052],
       [0.11060432, 0.11060432, 0.11060432],
       [0.1125    , 0.1125    , 0.1125    ],
       [0.10233992, 0.10233992, 0.10233992],
       [0.09681372, 0.09681372, 0.09681372],
       [0.08897059, 0.08897059, 0.08897059],
       [0.

In [15]:
print(np.shape(x_train_normal))

(1341, 256, 256, 3)


In [16]:
datagen_pneumonia = ImageDataGenerator(
    zoom_range = 0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip = True
)

datagen_pneumonia.fit(x_train_pneumonia)
batch_size = np.shape(x_train_pneumonia)[0]
x_train_pneumonia_1 = next(datagen_pneumonia.flow(x_train_pneumonia, batch_size=batch_size))
x_train_pneumonia_expanded = np.concatenate((np.array(x_train_pneumonia), np.array(x_train_pneumonia_1)), axis=0)

x_train_pneumonia = x_train_pneumonia_expanded

datagen_pneumonia.fit(x_train_pneumonia)
batch_size = np.shape(x_train_pneumonia)[0]
x_train_pneumonia_1 = next(datagen_pneumonia.flow(x_train_pneumonia, batch_size=batch_size))
x_train_pneumonia_expanded = np.concatenate((np.array(x_train_pneumonia), np.array(x_train_pneumonia_1)), axis=0)

In [17]:
print(np.shape(x_train_pneumonia_expanded))

(15500, 256, 256, 3)


In [18]:
datagen_normal = ImageDataGenerator(
    zoom_range = 0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip = True
)

datagen_normal.fit(x_train_normal)

batch_size = np.shape(x_train_normal)[0]
x_train_normal_1 = next(datagen_normal.flow(x_train_normal, batch_size=batch_size))
x_train_normal_expanded = np.concatenate((np.array(x_train_normal), np.array(x_train_normal_1)), axis=0)

x_train_normal = x_train_normal_1

for i in range(3):
    datagen_normal.fit(x_train_normal)
    x_train_normal_1 = next(datagen_normal.flow(x_train_normal, batch_size=batch_size))
    x_train_normal_expanded = np.concatenate((x_train_normal_expanded, np.array(x_train_normal_1)), axis=0)

    x_train_normal = x_train_normal_1
    
x_train_normal = x_train_normal[:1045]

datagen_normal.fit(x_train_normal)
batch_size = np.shape(x_train_normal)[0]
x_train_normal_1 = next(datagen_normal.flow(x_train_normal, batch_size=batch_size))
x_train_normal_expanded = np.concatenate((x_train_normal_expanded, np.array(x_train_normal_1)), axis=0)

x_train_normal = x_train_normal_expanded

datagen_normal.fit(x_train_normal)
batch_size = np.shape(x_train_normal)[0]
x_train_normal_1 = next(datagen_normal.flow(x_train_normal, batch_size=batch_size))
x_train_normal_expanded = np.concatenate((x_train_normal_expanded, np.array(x_train_normal_1)), axis=0)

In [19]:
print(np.shape(x_train_normal_expanded))

(15500, 256, 256, 3)


In [20]:
x_train = np.concatenate((x_train_pneumonia_expanded, x_train_normal_expanded), axis=0)

y_train = []
for i in range(len(x_train_pneumonia_expanded)):
    y_train.append(0)
for i in range(len(x_train_normal_expanded)):
    y_train.append(1)
y_train = np.array(y_train)

In [21]:
print(np.shape(y_train))

(31000,)


In [ ]:
# # Create the dictionary
# pneumonia_dict = {
#     'x_train': x_train,
#     'x_test': x_test,
#     'y_train': y_train,
#     'y_test': y_test,
#     'x_val': x_val,
#     'y_val': y_val
# }



In [56]:
# # Store the dictionary in a pickle file
# with open('Resources/pneumonia_dict.pkl', 'wb') as file:
#     pickle.dump(pneumonia_dict, file)

In [24]:
#!pip install h5py

In [22]:
import h5py

In [23]:
from tensorflow.keras.utils import to_categorical
# Assuming y_train contains labels 0 and 1
y_train = to_categorical(y_train, num_classes=2)

In [24]:
# We use transfer learning with VGG16 as the base model
def CNN_Model():
    base_model = VGG16(weights='imagenet', include_top = False, input_shape=(img_width, img_height, 3))
    # Freeze the base model
    for layer in base_model.layers:
        layer.trainable = False
    for i in range(3):
        base_model.layers[-2-i].trainable = True
    CNN = Sequential()
    CNN.add(Input(shape=(img_width, img_height,3)))
    CNN.add(base_model)
    CNN.add(Flatten())
    CNN.add(Dropout(0.3))
    CNN.add(Dense(128, activation='relu', kernel_regularizer=l2(0.05)))
    CNN.add(Dropout(0.2))
    CNN.add(Dense(2, activation='sigmoid'))
    return CNN
# Training The CNN
model = CNN_Model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()
# Visualized Layers of CNN
keras.utils.plot_model(model, show_shapes=True)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)                   │ (None, 8, 8, 512)           │      14,714,688 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 32768)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 32768)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 128)                 │       4,194,432 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 2)                   │             258 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 18,909,378 (72.13 MB)

 Trainable params: 11,274,114 (43.01 MB)

 Non-trainable params: 7,635,264 (29.13 MB)

You must install pydot (`pip install pydot`) for `plot_model` to work.


In [25]:
hist = model.fit(x_train, y_train, epochs= 6, validation_data= (x_val, y_val))
print(hist.history)
best_score = max(hist.history['val_accuracy'])
print(f"Best Validation score is: {best_score}")

Epoch 1/6
969/969 ━━━━━━━━━━━━━━━━━━━━ 2001s 2s/step - accuracy: 0.8517 - loss: 0.9575 - val_accuracy: 0.0625 - val_loss: 6.7882
Epoch 2/6
969/969 ━━━━━━━━━━━━━━━━━━━━ 1877s 2s/step - accuracy: 0.9717 - loss: 0.1307 - val_accuracy: 0.0625 - val_loss: 11.8690
Epoch 3/6
969/969 ━━━━━━━━━━━━━━━━━━━━ 6413s 7s/step - accuracy: 0.9792 - loss: 0.1198 - val_accuracy: 0.0000e+00 - val_loss: 7.7465
Epoch 4/6
969/969 ━━━━━━━━━━━━━━━━━━━━ 1879s 2s/step - accuracy: 0.9853 - loss: 0.0691 - val_accuracy: 0.0625 - val_loss: 8.6849
Epoch 5/6
969/969 ━━━━━━━━━━━━━━━━━━━━ 1861s 2s/step - accuracy: 0.9864 - loss: 0.0692 - val_accuracy: 0.0000e+00 - val_loss: 10.6234
Epoch 6/6
969/969 ━━━━━━━━━━━━━━━━━━━━ 1852s 2s/step - accuracy: 0.9890 - loss: 0.0625 - val_accuracy: 0.0000e+00 - val_loss: 9.0913
{'accuracy': [0.9254516363143921, 0.9721935391426086, 0.9790967702865601, 0.9841613173484802, 0.9864193797111511, 0.9884193539619446], 'loss': [0.37810757756233215, 0.12836380302906036, 0.10611982643604279, 0.074